In [1]:
import tempfile
import wget
import numpy as np
import matplotlib.pyplot as plt
import os

from azure.storage.blob import ContainerClient

modis_account_name = 'modissa'
modis_container_name = 'modis-006'
modis_account_url = 'https://' + modis_account_name + '.blob.core.windows.net/'
modis_blob_root = modis_account_url + modis_container_name + '/'

# This file is provided by NASA; it indicates the lat/lon extents of each
# MODIS tile.
#
# The file originally comes from:
#
# https://modis-land.gsfc.nasa.gov/pdf/sn_bound_10deg.txt
modis_tile_extents_url = modis_blob_root + 'sn_bound_10deg.txt'

temp_dir = os.path.join(tempfile.gettempdir(),'modis')
os.makedirs(temp_dir,exist_ok=True)
fn = os.path.join(temp_dir,modis_tile_extents_url.split('/')[-1])
wget.download(modis_tile_extents_url, fn)

# Load this file into a table, where each row is (v,h,lonmin,lonmax,latmin,latmax)
modis_tile_extents = np.genfromtxt(fn,
                                   skip_header = 7,
                                   skip_footer = 3)

modis_container_client = ContainerClient(account_url=modis_account_url,
                                         container_name=modis_container_name,
                                         credential=None)

In [3]:
def lat_lon_to_modis_tile(lat,lon):
    """
    Get the modis tile indices (h,v) for a given lat/lon

    https://www.earthdatascience.org/tutorials/convert-modis-tile-to-lat-lon/
    """

    found_matching_tile = False
    i = 0
    while(not found_matching_tile):
        found_matching_tile = lat >= modis_tile_extents[i, 4] \
                              and lat <= modis_tile_extents[i, 5] \
                              and lon >= modis_tile_extents[i, 2] and lon <= modis_tile_extents[i, 3]
        i += 1

    v = int(modis_tile_extents[i-1, 0])
    h = int(modis_tile_extents[i-1, 1])

    return h,v


def list_blobs_in_folder(container_name,folder_name):
    """
    List all blobs in a virtual folder in an Azure blob container
    """

    files = []
    generator = modis_container_client.list_blobs(name_starts_with=folder_name)
    for blob in generator:
        files.append(blob.name)
    return files


def list_hdf_blobs_in_folder(container_name,folder_name):
    """"
    List .hdf files in a folder
    """

    files = list_blobs_in_folder(container_name,folder_name)
    files = [fn for fn in files if fn.endswith('.hdf')]
    return files

In [19]:
import geopandas as gpd
cells = gpd.read_file('data/grid_cells.geojson')


In [43]:
tiles = []
for i, row in cells.iterrows():
    if i == 0:
        print(row.geometry.centroid.x, row.geometry.centroid.y)
    row['x'] = row.geometry.centroid.x
    row['y'] = row.geometry.centroid.y
    tiles.append(lat_lon_to_modis_tile(row['y'], row['x']))

-118.7234447492404 37.07777576986577


In [24]:
tiles = np.array(tiles)
tiles = np.unique(tiles, axis=0)

In [25]:
tiles

array([[7, 5],
       [8, 4],
       [8, 5],
       [9, 4]])

In [55]:
test_tile = lat_lon_to_modis_tile(37.077, -118.72)

In [84]:
test_tile

(7, 5)

In [78]:
product = 'MOD10A1'

daynum = '2019135'
folder = product + '/' + '{:0>2d}/{:0>2d}'.format(test_tile[0],test_tile[1]) + '/' + daynum

# Find all HDF files from this tile on this day
filenames = list_hdf_blobs_in_folder(modis_container_name,folder)
print('Found {} matching file(s):'.format(len(filenames)))
for fn in filenames:
    print(fn)

# Work with the first returned URL
blob_name = filenames[0]

# Download to a temporary file
url = modis_blob_root + blob_name

filename = os.path.join(temp_dir,blob_name.replace('/','_'))
if not os.path.isfile(filename):
    wget.download(url,filename)


Found 1 matching file(s):
MOD10A1/07/05/2019135/MOD10A1.A2019135.h07v05.006.2019149220243.hdf


In [93]:
import rioxarray as rxr
import rasterio as rio
ds = rxr.open_rasterio(filename)
list(ds.keys())


['NDSI_Snow_Cover',
 'NDSI_Snow_Cover_Basic_QA',
 'NDSI_Snow_Cover_Algorithm_Flags_QA',
 'NDSI',
 'Snow_Albedo_Daily_Tile',
 'orbit_pnt',
 'granule_pnt']

In [94]:
type(ds)

xarray.core.dataset.Dataset

In [124]:
img = ds['NDSI_Snow_Cover']
img_flags = ds['NDSI_Snow_Cover_Algorithm_Flags_QA']

In [125]:
img


<xarray.DataArray 'NDSI_Snow_Cover' (band: 1, y: 2400, x: 2400)>
[5760000 values with dtype=uint8]
Coordinates:
  * y            (y) float64 4.448e+06 4.447e+06 ... 3.337e+06 3.336e+06
  * x            (x) float64 -1.223e+07 -1.223e+07 ... -1.112e+07 -1.112e+07
  * band         (band) int32 1
    spatial_ref  int32 0
Attributes:
    _FillValue:    255.0
    scale_factor:  1.0
    add_offset:    0.0
    long_name:     NDSI snow cover from best observation of the day
    units:         none

In [126]:
val= img.sel(x=-11959000.0, y=3653000.0, method='nearest')
val_f = img_flags.sel(x=-11959000.0, y=3653000.0, method='nearest')

In [127]:
print(val)
print(val_f)

<xarray.DataArray 'NDSI_Snow_Cover' (band: 1)>
array([239], dtype=uint8)
Coordinates:
    y            float64 3.653e+06
    x            float64 -1.196e+07
  * band         (band) int32 1
    spatial_ref  int32 0
Attributes:
    _FillValue:    255.0
    scale_factor:  1.0
    add_offset:    0.0
    long_name:     NDSI snow cover from best observation of the day
    units:         none
<xarray.DataArray 'NDSI_Snow_Cover_Algorithm_Flags_QA' (band: 1)>
array([239], dtype=uint8)
Coordinates:
    y            float64 3.653e+06
    x            float64 -1.196e+07
  * band         (band) int32 1
    spatial_ref  int32 0
Attributes:
    _FillValue:    255.0
    scale_factor:  1.0
    add_offset:    0.0
    long_name:     NDSI snow cover algorithm bit flags
    units:         none
